In [20]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [21]:
!pip install transformers
!pip install sentencepiece
!pip install datasets
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
from datasets import load_dataset
data = []
label = []
XY = []
    
dataset = load_dataset('c4', 'realnewslike', split='train', streaming=True)
train_stream = dataset
for i, item in enumerate(train_stream):
    if i >= 10**2:
        break
    
    XY.append({'text': item['text'], 'label': item['url']})

    data.append(item['text'])
    label.append(item['url'])
    print(item)

k = data

{'text': 'After the martyrdom of St. Boniface, Vergilius was made Bishop of Salzburg (766 or 767) and laboured successfully for the upbuilding of his diocese as well as for the spread of the Faith in neighbouring heathen countries, especially in Carinthia. He died at Salzburg, 27 November, 789. In 1233 he was canonized by Gregory IX. His doctrine that the earth is a sphere was derived from the teaching of ancient geographers, and his belief in the existence of the antipodes was probably influenced by the accounts which the ancient Irish voyagers gave of their journeys. This, at least, is the opinion of Rettberg ("Kirchengesch. Deutschlands", II, 236).', 'timestamp': '2019-04-22T08:07:02Z', 'url': 'https://www.catholic.org/encyclopedia/view.php?id=11981'}
{'text': '"Whoever gets him, they\'ll be getting a good one," David Montgomery said.\nINDIANAPOLIS — Hakeem Butler has been surrounded by some of the best wide receivers on the planet this week at the NFL Scouting Combine.\nIt’s an exp

In [33]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer 

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']
        label = item['label']
        encoded_text = self.tokenizer.encode_plus(
            text,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoded_text['input_ids'].squeeze(),
            'attention_mask': encoded_text['attention_mask'].squeeze(),
            #'label': torch.tensor(label)
        }


In [34]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [35]:
custom_dataset = CustomDataset(XY, tokenizer)
custom_dataloader = DataLoader(custom_dataset, batch_size=8, shuffle=True)

for batch in custom_dataloader:
    print(batch)

{'input_ids': tensor([[   71,  3570,  2243,  ...,     0,     0,     0],
        [12077,    23,     9,  ...,     0,     0,     0],
        [  100,    47, 14317,  ...,   150,  2829,     1],
        ...,
        [  332,    83,     7,  ...,   135,     5,     1],
        [  283,   439,  6698,  ...,  2776, 26897,     1],
        [    3, 24833,  4611,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])}
{'input_ids': tensor([[ 2150,    12, 31884,  ...,     0,     0,     0],
        [  621,     3,    60,  ...,     0,     0,     0],
        [16637,  4328,    49,  ...,     0,     0,     0],
        ...,
        [ 6902,    55,    31,  ...,     0,     0,     0],
        [ 5097,     5,   446,  ...,     0,     0,     0],
        [ 1980,  8041,    10,  ...,   446,   434,     1]]), 'attentio

In [36]:
inputs = tokenizer(k, truncation=True, padding=True, return_tensors='pt')
attention_mask = inputs['attention_mask']
input_ids11 = inputs['input_ids']
batch_size = 4
data = torch.utils.data.TensorDataset(inputs['input_ids'], attention_mask)
loader = torch.utils.data.DataLoader(data, batch_size=batch_size)

In [37]:
attention_mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])

In [38]:
from transformers import T5ForConditionalGeneration, AdamW

model = T5ForConditionalGeneration.from_pretrained('t5-small')
optimizer = AdamW(model.parameters(), lr=1e-4)

epochs = 5
best_score, ind = 100, 100
for epoch in range(epochs):
    for batch in loader:
        inputs, attention_mask = batch
        labels = inputs.clone()
        labels[labels == tokenizer.pad_token_id] = -100

        outputs = model(inputs, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if loss.item() < best_score:
        best_score = loss.item()
        ind = epoch + 1

    #if epoch % 10 == 4:
    print(f'Epoch {epoch+1}: loss = {loss.item()}')


Epoch 1: loss = 0.5824218988418579
Epoch 2: loss = 0.28606116771698
Epoch 3: loss = 0.19685421884059906
Epoch 4: loss = 0.14653554558753967
Epoch 5: loss = 0.10518614202737808


In [ ]:
valid = []
label = []
data = []

dataset = load_dataset('c4', 'realnewslike', split='validation', streaming=True)
train_stream = dataset
for i, item in enumerate(train_stream):
    if i >= 10**2:
        break

    valid.append({'text': item['text'], 'label': item['url']})
    data.append(item['text'])
    label.append(item['url'])
    print(item)


In [41]:
labels = tokenizer(label, truncation=True, padding=True, return_tensors='pt')
data = tokenizer(data, truncation=True, padding=True, return_tensors='pt')

In [42]:
q = labels['input_ids']
qq = labels['attention_mask']
i = data['input_ids']
ii = data['attention_mask']

In [43]:
inputs = torch.tensor(i)
input_mask = torch.tensor(ii)
labels = torch.tensor(q)
label_mask = torch.tensor(qq)

batch_size = 8
data = torch.utils.data.TensorDataset(inputs,input_mask, labels, label_mask)
loader = torch.utils.data.DataLoader(data, batch_size=batch_size)

<ipython-input-43-bb732778084c>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(i)
<ipython-input-43-bb732778084c>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_mask = torch.tensor(ii)
<ipython-input-43-bb732778084c>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(q)
<ipython-input-43-bb732778084c>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourc

In [44]:
from transformers import T5ForConditionalGeneration

# model = T5ForConditionalGeneration.from_pretrained('t5-small')

total_loss = 0
num_examples = 0

for batch in loader:
    inputs, input_mask, labels, label_mask = batch

    outputs = model(inputs, attention_mask=input_mask, labels=labels, 
                    decoder_attention_mask=label_mask)
    loss = outputs[0]

    total_loss += loss.item() * inputs.shape[0]
    num_examples += inputs.shape[0]

average_loss = total_loss / num_examples

print(f'Average validation loss: {average_loss:.4f}')


Average validation loss: 6.9407


In [45]:
import sacrebleu

def compute_bleu(model, loader):
    references = []
    hypotheses = []

    for batch in loader:
        inputs, input_mask, labels, label_mask = batch

        outputs = model.generate(inputs, attention_mask=input_mask, max_length=50)
        decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        hypotheses.extend(decoded_outputs)
        references.extend(decoded_labels)

    bleu = sacrebleu.corpus_bleu(hypotheses, [references])

    return bleu.score

bleu_score = compute_bleu(model, loader)

print(f'BLEU score: {bleu_score:.4f}')


BLEU score: 0.0335
